Now that we know which models are performing better, it's time to perform cross validation and tune hyperparameters.
- Do a google search for hyperparameter ranges for each type of model.

GridSearch/RandomSearch are a great methods for checking off both of these tasks.
- BUT we have a problem - if we calculated a numerical value to encode city (such as the mean of sale prices in that city) on the training data, we can't cross validate 
- The rows in each validation fold were part of the original calculation of the mean for that city - that means we're leaking information!
- While sklearn's built in functions are extremely useful, sometimes it is necessary to do things ourselves

You need to create two functions to replicate what Gridsearch does under the hood

**`custom_cross_validation()`**
- Should take the training data, and divide it into multiple train/validation splits. 
- Look into `sklearn.model_selection.KFold` to accomplish this - the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) shows how to split a dataframe and loop through the indexes of your split data. 
- Within your function, you should compute the city means on the training folds just like you did in Notebook 1 - you may have to re-join the city column to do this - and then join these values to the validation fold

**`hyperparameter_search()`**
- Should take the validation and training splits from your previous function, along with your dictionary of hyperparameter values
- For each set of hyperparameter values, fit your chosen model on each set of training folds, and take the average of your chosen scoring metric. [itertools.product()](https://docs.python.org/3/library/itertools.html) will be helpful for looping through all combinations of hyperparameter values
- Your function should output the hyperparameter values corresponding the highest average score across all folds. Alternatively, it could also output a model object fit on the full training dataset with these parameters.

Docstrings have been provided below to get you started. Once you're done developing your functions, you should move them to `functions_variables.py` to keep your notebook clean 

Bear in mind that these instructions are just one way to tackle this problem - the inputs and output formats don't need to be exactly as specified here.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
import itertools
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
def custom_cross_validation(training_data, n_splits=5):
    '''Creates n_splits sets of training and validation folds

    Args:
      training_data: the dataframe of features and target to be divided into folds
      n_splits: the number of sets of folds to be created

    Returns:
      A tuple of lists, where the first index is a list of the training folds, 
      and the second the corresponding validation fold
    '''
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    training_folds = []
    validation_folds = []

    for train_index, val_index in kf.split(training_data):
        train_fold = training_data.iloc[train_index]
        val_fold = training_data.iloc[val_index]

        # Compute city means on the training fold
        city_means = train_fold.groupby('location.address.city')['description.sold_price'].mean()
        city_means = city_means.to_frame().rename(columns={'description.sold_price': 'city_mean_price'}).reset_index()

        # Merge city means with the validation fold
        val_fold = val_fold.merge(city_means, on='location.address.city', how='left')
        train_fold = train_fold.merge(city_means, on='location.address.city', how='left')

        training_folds.append(train_fold)
        validation_folds.append(val_fold)

    return training_folds, validation_folds

In [3]:
def hyperparameter_search(training_folds, validation_folds, param_grid):
    '''Outputs the best combination of hyperparameter settings in the param grid, 
    given the training and validation folds

    Args:
      training_folds: the list of training fold dataframes
      validation_folds: the list of validation fold dataframes
      param_grid: the dictionary of possible hyperparameter values for the chosen model

    Returns:
      A list of the best hyperparameter settings based on the chosen metric
    '''
    best_params = None
    best_score = float('inf')

    # Generate all combinations of hyperparameter values
    param_combinations = list(itertools.product(*param_grid.values()))

    for params in param_combinations:
        param_dict = dict(zip(param_grid.keys(), params))
        scores = []

        for train_fold, val_fold in zip(training_folds, validation_folds):
            model = RandomForestRegressor(**param_dict)
            X_train, y_train = train_fold.drop(columns=['description.sold_price']), train_fold['description.sold_price']
            X_val, y_val = val_fold.drop(columns=['description.sold_price']), val_fold['description.sold_price']

            model.fit(X_train, y_train)
            y_pred = model.predict(X_val)
            score = mean_squared_error(y_val, y_pred)
            scores.append(score)

        avg_score = np.mean(scores)

        if avg_score < best_score:
            best_score = avg_score
            best_params = param_dict

    return best_params

In [ ]:
# example usage
train_folds, val_folds = custom_cross_validation(df, n_splits=5)

In [ ]:
# example usage
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}
best_params = hyperparameter_search(train_folds, val_folds, param_grid)
print("Best Hyperparameters:", best_params)

We want to make sure that we save our models.  In the old days, one just simply pickled (serialized) the model.  Now, however, certain model types have their own save format.  If the model is from sklearn, it can be pickled, if it's xgboost, for example, the newest format to save it in is JSON, but it can also be pickled.  It's a good idea to stay with the most current methods. 
- you may want to create a new `models/` subdirectory in your repo to stay organized

In [ ]:
# Random Forest
# Initialize the model
rf_model_all = RandomForestRegressor(random_state=42)

# Train the model
rf_model_all.fit(X_train, y_train)

# Predict on the test set
y_pred_rf_all = rf_model_all.predict(X_test)

# Evaluate Random Forest model
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = mean_squared_error(y_test, y_pred_rf, squared=False)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("\n\nRandom Forest Metrics:")
print(f"MSE: {mse_rf}")
print(f"RMSE: {rmse_rf}")
print(f"MAE: {mae_rf}")
print(f"R²: {r2_rf}")

In [ ]:
# Refit models with selected features using Lasso
# Ensure `selected_features` is a boolean array
selected_features = np.array(lasso.coef_ != 0)
X_train_selected = X_train_scaled[:, selected_features]
X_test_selected = X_test_scaled[:, selected_features]

# Random Forest
# Initialize the model
rf_model_lasso = RandomForestRegressor(random_state=42)

# Train the model
rf_model_lasso.fit(X_train_selected, y_train)

# Predict on the test set
y_pred_rf_lasso = rf_model_lasso.predict(X_test_selected)

# Evaluate Random Forest model
mse_rf_lasso = mean_squared_error(y_test, y_pred_rf_lasso)
rmse_rf_lasso = mean_squared_error(y_test, y_pred_rf_lasso, squared=False)
mae_rf_lasso = mean_absolute_error(y_test, y_pred_rf_lasso)
r2_rf_lasso = r2_score(y_test, y_pred_rf_lasso)

print("\n\nRandom Forest Metrics with Lasso Regularization:")
print(f"MSE: {mse_rf_lasso}")
print(f"RMSE: {rmse_rf_lasso}")
print(f"MAE: {mae_rf_lasso}")
print(f"R²: {r2_rf_lasso}")

In [ ]:
# Refit models with selected features using Recursive Feature Elimination (RFE)
# Ensure `selected_features` is a boolean array
selected_features_rfe = np.array(rfe.support_)
X_train_selected_rfe = X_train_scaled[:, selected_features_rfe]
X_test_selected_rfe = X_test_scaled[:, selected_features_rfe]

# Random Forest
# Initialize the model
rf_model_rfe = RandomForestRegressor(random_state=42)

# Train the model
rf_model_rfe.fit(X_train_selected_rfe, y_train)

# Predict on the test set
y_pred_rf_rfe = rf_model_rfe.predict(X_test_selected_rfe)

# Evaluate Random Forest model
mse_rf_rfe = mean_squared_error(y_test, y_pred_rf_rfe)
rmse_rf_rfe = mean_squared_error(y_test, y_pred_rf_rfe, squared=False)
mae_rf_rfe = mean_absolute_error(y_test, y_pred_rf_rfe)
r2_rf_rfe = r2_score(y_test, y_pred_rf_rfe)

print("\n\nRandom Forest Metrics with RFE Regularization:")
print(f"MSE: {mse_rf_rfe}")
print(f"RMSE: {rmse_rf_rfe}")
print(f"MAE: {mae_rf_rfe}")
print(f"R²: {r2_rf_rfe}")

Once you've identified which model works the best, implement a prediction pipeline to make sure that you haven't leaked any data, and that the model could be easily deployed if desired.
- Your pipeline should load the data, process it, load your saved tuned model, and output a set of predictions
- Assume that the new data is in the same JSON format as your original data - you can use your original data to check that the pipeline works correctly
- Beware that a pipeline can only handle functions with fit and transform methods.
- Classes can be used to get around this, but now sklearn has a wrapper for user defined functions.
- You can develop your functions or classes in the notebook here, but once they are working, you should import them from `functions_variables.py` 

In [ ]:
# Build pipeline here

Pipelines come from sklearn.  When a pipeline is pickled, all of the information in the pipeline is stored with it.  For example, if we were deploying a model, and we had fit a scaler on the training data, we would want the same, already fitted scaling object to transform the new data with.  This is all stored when the pipeline is pickled.
- save your final pipeline in your `models/` folder

In [ ]:
# save your pipeline here